https://mp.weixin.qq.com/s/3BnOTC8BmPEp5zBx6HXD8Q

The error ModuleNotFoundError: No module named 'numpy.distutils' occurs when using Faiss with Python 3.12. This is because the numpy.distutils module has been deprecated and removed in Python 3.12
1
.

使用py3.8环境 85 server

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import metrics
import autosklearn.regression

In [ ]:
X, y = datasets.load_boston(return_X_y=True)

In [5]:
X.shape

(506, 13)

In [12]:
X.dtype

dtype('float64')

In [11]:
feature_types = (['numerical'] * 3) + ['categorical'] + (['numerical'] * 9)
feature_types

['numerical',
 'numerical',
 'numerical',
 'categorical',
 'numerical',
 'numerical',
 'numerical',
 'numerical',
 'numerical',
 'numerical',
 'numerical',
 'numerical',
 'numerical']

In [6]:
y.shape

(506,)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,  # 此任务的最长时间 秒
    per_run_time_limit=30,  # 每个模型评估的时间
    tmp_folder="./tmp",
    memory_limit=10240,  # 设置内存限制为10GB
    n_jobs=30,  # 设置并行任务数量 
)

In [15]:
automl

AutoSklearnRegressor(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                     memory_limit=10240, per_run_time_limit=30,
                     time_left_for_this_task=120, tmp_folder='./tmp')

In [16]:
automl.fit(X_train, y_train, dataset_name='boston', feat_type=feature_types) 

[WARNING] [2025-03-18 21:07:07,322:Client-AutoMLSMBO(1)::boston] Dataset boston already in meta-data. Removing occurence.


AutoSklearnRegressor(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                     memory_limit=10240, per_run_time_limit=30,
                     time_left_for_this_task=120, tmp_folder='./tmp')

In [17]:
# sprint_statistics()函数总结了上述搜索和选择的最佳模型的性能
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: boston
  Metric: r2
  Best validation score: 0.879861
  Number of target algorithm runs: 12
  Number of successful target algorithm runs: 9
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0



- 数据集名称：
Dataset name: boston
表示使用的数据集是波士顿房价数据集（boston），这是一个经典的回归任务数据集。

- 评估指标：
Metric: r2
使用的评估指标是 R²（决定系数），这是回归任务中常用的指标，用于衡量模型对数据的拟合程度。R² 的值越接近 1，表示模型的拟合效果越好。

- 最佳验证分数：
Best validation score: 0.879861
表示在验证集上，最佳模型的 R² 值为 0.879861。这意味着该模型对数据的拟合效果较好，能够解释约 88% 的数据变异。

- 目标算法运行次数：
Number of target algorithm runs: 12
表示 auto-sklearn 共运行了 12 次目标算法（即尝试了 12 种不同的模型或模型配置）。

- 成功运行次数：
Number of successful target algorithm runs: 9
表示在这 12 次运行中，有 9 次成功完成，没有出现错误或异常。

- 崩溃运行次数：
Number of crashed target algorithm runs: 0
表示没有目标算法运行失败（即没有崩溃）。

- 超出时间限制的运行次数：
Number of target algorithms that exceeded the time limit: 3
表示有 3 次运行因为超出了时间限制而被终止。这可能是因为某些模型或配置在给定的时间内无法完成训练。

- 超出内存限制的运行次数：
Number of target algorithms that exceeded the memory limit: 0
表示没有运行因为超出内存限制而被终止。

此外，还有其他参数，如n_jobs（并行作业数）、ensemble_size、initial_configurations_via_metalearning，可用于微调分类器。默认情况下，上述搜索命令会创建一组表现最佳的模型。为了避免过度拟合，我们可以通过更改设置ensemble_size = 1和initial_configurations_via_metalearning = 0来禁用它。

In [18]:
# 搜索考虑的所有模型打印排行榜
print(automl.leaderboard())

          rank  ensemble_weight               type      cost   duration
model_id                                                               
3            1             0.74   gaussian_process  0.120139  12.362904
10           2             0.02   gaussian_process  0.144066  10.659464
12           3             0.24  gradient_boosting  0.161145   1.396197


 `leaderboard()` 输出显示了 `auto-sklearn` 在搜索过程中考虑的所有模型的排行榜。这个排行榜根据模型的性能（如验证分数）对模型进行了排序，并提供了每个模型的相关信息。以下是对这些列的详细解释：

### 列的含义
1. **rank**：
   - 表示模型在排行榜中的排名。排名越靠前，表示模型的性能越好。

2. **ensemble_weight**：
   - 表示该模型在最终集成模型中的权重。权重越高，说明该模型在集成模型中越重要。

3. **type**：
   - 表示模型的类型。例如：
     - `gaussian_process`：高斯过程回归模型。
     - `gradient_boosting`：梯度提升模型。

4. **cost**：
   - 表示模型的损失值（越低越好）。例如，对于回归任务，这可能是均方误差（MSE）或其他损失函数的值。

5. **duration**：
   - 表示训练该模型所花费的时间（单位为秒）。

### 结果解读
从您提供的排行榜来看：
- **模型 3**：
  - 排名第 1 位。
  - 是一个高斯过程回归模型（`gaussian_process`）。
  - 在集成模型中的权重为 0.74，说明它在集成模型中占据了较大的比重。
  - 损失值为 0.120139，说明它的性能较好。
  - 训练时间为 12.36 秒。

- **模型 10**：
  - 排名第 2 位。
  - 也是一个高斯过程回归模型。
  - 在集成模型中的权重为 0.02，说明它对最终结果的贡献较小。
  - 损失值为 0.144066，略高于模型 3。
  - 训练时间为 10.66 秒。

- **模型 12**：
  - 排名第 3 位。
  - 是一个梯度提升模型（`gradient_boosting`）。
  - 在集成模型中的权重为 0.24，说明它对最终结果有一定的贡献。
  - 损失值为 0.161145，略高于前两个模型。
  - 训练时间最短，仅为 1.40 秒。

### 分析与建议
1. **高斯过程回归模型表现较好**：
   - 在前两名中，高斯过程回归模型的性能优于梯度提升模型，这可能是因为高斯过程回归在这种数据集上更适合。

2. **模型权重与性能的关系**：
   - 模型 3 的权重最高，损失值最低，说明它在集成模型中起到了主导作用。
   - 模型 12 虽然损失值略高，但由于其训练时间短，可能在某些情况下也有一定的优势。

3. **进一步优化**：
   - 如果您希望进一步提高模型性能，可以尝试以下方法：
     - 增加 `time_left_for_this_task` 参数的值，以允许更长时间的搜索。
     - 调整 `per_run_time_limit` 参数，为每次运行提供更多时间。
     - 对数据集进行更细致的预处理，例如特征工程，以提高模型性能。

4. **模型选择**：
   - 如果您需要一个单一的最佳模型，可以查看 `automl.show_models()` 的输出，了解最终选择的模型及其配置。
   - 如果您需要更详细的交叉验证结果，可以查看 `automl.cv_results_`。

In [19]:
# 打印有关所考虑模型的信息
print(automl.show_models())

{3: {'model_id': 3, 'rank': 1, 'cost': 0.12013883058064923, 'ensemble_weight': 0.74, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x724bc3ef6fd0>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x724bde6ad9d0>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x724bde6adcd0>, 'sklearn_regressor': GaussianProcessRegressor(alpha=0.037731974209709904,
                         kernel=RBF(length_scale=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
                         n_restarts_optimizer=10, normalize_y=True,
                         random_state=1)}, 10: {

`auto-sklearn` 的 `show_models()` 方法提供了关于所考虑模型的详细信息，包括模型的配置、预处理器和超参数设置。从您提供的输出中，我们可以看到每个模型的详细信息，以下是对这些信息的解读：

### 模型信息解读

#### 模型 3
- **模型 ID**：3
- **排名**：1（最佳模型）
- **损失值**：0.12013883058064923（越低越好）
- **集成权重**：0.74（在集成模型中占据较大权重）
- **数据预处理器**：`DataPreprocessorChoice`（具体配置未显示）
- **特征预处理器**：`FeaturePreprocessorChoice`（具体配置未显示）
- **回归器**：`RegressorChoice`（具体配置未显示）
- **Sklearn 回归器**：`GaussianProcessRegressor`（高斯过程回归器）
  - **超参数**：
    - `alpha=0.037731974209709904`：噪声水平。
    - `kernel=RBF`：径向基函数（RBF）核。
    - `n_restarts_optimizer=10`：优化器重新启动的次数。
    - `normalize_y=True`：是否对目标值进行归一化。
    - `random_state=1`：随机种子。

#### 模型 10
- **模型 ID**：10
- **排名**：2
- **损失值**：0.14406582457594364
- **集成权重**：0.02（权重较低，对集成模型贡献较小）
- **数据预处理器**：`DataPreprocessorChoice`
- **特征预处理器**：`FeaturePreprocessorChoice`
- **回归器**：`RegressorChoice`
- **Sklearn 回归器**：`GaussianProcessRegressor`（高斯过程回归器）
  - **超参数**：
    - `alpha=0.017488165983844263`：噪声水平。
    - `kernel=RBF`：径向基函数核。
    - `n_restarts_optimizer=10`：优化器重新启动的次数。
    - `normalize_y=True`：是否对目标值进行归一化。
    - `random_state=1`：随机种子。

#### 模型 12
- **模型 ID**：12
- **排名**：3
- **损失值**：0.16114495854950772
- **集成权重**：0.24
- **数据预处理器**：`DataPreprocessorChoice`
- **特征预处理器**：`FeaturePreprocessorChoice`
- **回归器**：`RegressorChoice`
- **Sklearn 回归器**：`HistGradientBoostingRegressor`（直方图梯度提升回归器）
  - **超参数**：
    - `l2_regularization=0.0018018055158809108`：L2 正则化强度。
    - `learning_rate=0.05048450688348591`：学习率。
    - `max_iter=512`：最大迭代次数。
    - `max_leaf_nodes=3`：最大叶子节点数。
    - `min_samples_leaf=1`：叶子节点最小样本数。
    - `n_iter_no_change=19`：连续多少次迭代没有改进时停止训练。
    - `validation_fraction=0.19819612428903174`：用于提前停止的验证集比例。
    - `warm_start=True`：是否在现有模型基础上继续训练。

### 分析与建议
1. **模型选择**：
   - 模型 3 是最佳模型，使用了高斯过程回归器，其损失值最低，权重最高。
   - 模型 10 和 12 也表现不错，但权重较低。

2. **超参数调整**：
   - 如果您希望进一步优化模型，可以尝试调整超参数。例如：
     - 对于高斯过程回归器，可以调整 `alpha` 和 `kernel` 参数。
     - 对于梯度提升回归器，可以调整 `learning_rate`、`max_iter` 和 `max_leaf_nodes` 等参数。

3. **特征预处理**：
   - `auto-sklearn` 自动选择数据预处理器和特征预处理器，但您也可以手动指定这些组件，以进一步优化模型性能。

4. **集成模型**：
   - `auto-sklearn` 使用集成方法将多个模型组合在一起。如果需要，您可以查看最终的集成模型配置，了解哪些模型被选中以及它们的权重。

5. **进一步探索**：
   - 如果您对某个特定模型感兴趣，可以使用 `auto-sklearn` 提供的 `get_model_with_configuration` 方法获取该模型的具体配置，并单独训练和评估它。

In [23]:
predictions = automl.predict(X_test) 
print("R2 score:", metrics.r2_score(y_test, predictions))

R2 score: 0.8750097787245696
